The program is referenced and modified from:
> https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5  

Reference article explaining how to improve the program:
> https://david-exiga.medium.com/music-generation-using-lstm-neural-networks-44f6780a4c5  

Additional Chinese program explanation:
> https://github.com/xitu/gold-miner/blob/master/TODO1/how-to-generate-music-using-a-lstm-neural-network-in-keras.md

In [1]:
# Install dependency 
# music21 Introduction: https://juejin.cn/post/7063827463058489352
! pip install music21 keras tensorflow[and-cuda] 


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
# For reading files
import glob
# array processing
import numpy
from matplotlib import pyplot
# keras for building deep learning model
import keras
from keras.models import Sequential
from keras.layers import Dense, TimeDistributed
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint


### Get all notes and chords from midi files in a directory

In [3]:
"""
Extract all notes and chords from MIDI files in a directory.

This script uses music21 to process MIDI files. It reads all `.mid` files in the 
specified directory, extracts notes and chords, and appends them to the `notes` list.
"""

# Import necessary modules from music21 for MIDI processing
from music21 import converter, instrument, note, chord
import glob

# List to store all notes and chords from the MIDI files
notes = []

# Specify the path to the MIDI files (modify as needed)
midi_path = "./midi_songs/*.mid"

# Loop through all MIDI files in the specified directory
for file in glob.glob(midi_path):
    print(f"Parsing {file}")
    
    # Parse the MIDI file using music21
    midi = converter.parse(file)
    
    # Initialize a variable to hold notes and chords to be parsed
    notes_to_parse = None

    try:
        # If the MIDI file contains instrument parts, extract the first part
        s2 = instrument.partitionByInstrument(midi)
        notes_to_parse = s2.parts[0].recurse()  # Access notes recursively
    except AttributeError:
        # If no instrument parts, use the flat structure to access notes
        notes_to_parse = midi.flat.notes

    # Extract notes and chords from the parsed MIDI data
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            # If the element is a Note, extract its pitch as a string
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            # If the element is a Chord, extract its normal order as a string
            notes.append('.'.join(str(n) for n in element.normalOrder))

# Output the total number of notes and chords extracted
print(f"Total notes and chords extracted: {len(notes)}")


Parsing ./midi_songs/1.mid
Parsing ./midi_songs/2.mid
Parsing ./midi_songs/3.mid
Parsing ./midi_songs/4.mid
Parsing ./midi_songs/5.mid
Parsing ./midi_songs/6.mid
Parsing ./midi_songs/Ghosthack-AC22_Melody Loop Blue_Em 120BPM.mid
Parsing ./midi_songs/Ghosthack-AC22_Melody Loop Questions_Gm 126BPM.mid
Parsing ./midi_songs/Ghosthack-AC22_Melody Loop Closer_Fm 125BPM.mid
Parsing ./midi_songs/Ghosthack-AC22_Melody Loop Aurora_Am 140BPM.mid
Parsing ./midi_songs/Ghosthack-AC22_Melody Loop Alone_Dm 126BPM.mid
Parsing ./midi_songs/Ghosthack-AC22_Melody Loop Turbo_Fm 130BPM.mid
Parsing ./midi_songs/Ghosthack-AC22_Melody Loop Stay_Cm 126BPM.mid
Parsing ./midi_songs/Ghosthack-AC22_Melody Loop Elements_Fm 126BPM.mid
Parsing ./midi_songs/Ghosthack-AC22_Melody Loop Waterfall_Fm 140BPM.mid
Parsing ./midi_songs/Ghosthack-AC22_Melody Loop Lovesick_Gm 126BPM.mid
Parsing ./midi_songs/Ghosthack-AC22_Melody Loop Crusader_Cm 140BPM.mid
Parsing ./midi_songs/Ghosthack-AC22_Melody Loop Future_Am 126BPM.mid
Pars

### Prepare input and output for neural network use

In [4]:
# Prepare inputs and outputs for neural network

# Get the number of unique note names
n_vocab = len(set(notes))

# Get the sorted list of unique note names
pitchnames = sorted(set(item for item in notes))

# Create a dictionary mapping each note to a corresponding numeric ID (e.g., C4 -> 25)
note_to_int = {note: number for number, note in enumerate(pitchnames)}

print("\n===== Explanation of Variables =====\n")
print("notes: A list containing all the musical notes as strings.")
print(f"Total number of notes in the score: {len(notes)}")
print(f"Total unique note types in the score: {n_vocab}")
print(f"Unique note types: {pitchnames}")
print(f"Mapping of note types to IDs: {note_to_int}")

# Length of the input sequence for training
sequence_length = 100

# Create input and output sequences
network_input = []
network_output = []

# Ensure the notes list is long enough for the sequence length
if len(notes) > sequence_length:
    for i in range(len(notes) - sequence_length):
        # Input sequence of notes
        sequence_in = notes[i:i + sequence_length]
        # Corresponding output note
        sequence_out = notes[i + sequence_length]

        # Convert input sequence to numeric format
        network_input.append([note_to_int[char] for char in sequence_in])
        # Convert output note to numeric format
        network_output.append(note_to_int[sequence_out])

    print("\n===================\n")
    print(f"Total notes: {len(notes)}")
    print(f"Each {sequence_length} notes are converted into a training data set.")
    print(f"network_input: {len(network_input)} sequences, each containing {len(network_input[0])} numeric IDs.")
    print(f"network_output: {len(network_output)} numeric IDs, each corresponding to the next note in the sequence.")
    print("\n===================\n")
    print("Notes from index sequence_length - 10 to sequence_length:")
    print(notes[sequence_length-10:sequence_length])
    print("Corresponding numeric IDs:")
    print([note_to_int[char] for char in notes[sequence_length-10:sequence_length]])
    print("")
    print(f"Last 10 IDs of the 0th sequence in network_input: {network_input[0][sequence_length-10:sequence_length]}")
    print(f"Last 10 IDs of the 1st sequence in network_input: {network_input[1][sequence_length-10:sequence_length]}")
    print(f"Last 10 IDs of the 2nd sequence in network_input: {network_input[2][sequence_length-10:sequence_length]}")
    print("First three outputs in network_output:", network_output[0:3])

    # Number of patterns
    n_patterns = len(network_input)

    # Reshape input for LSTM compatibility
    normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))

    # Normalize input
    normalized_input = normalized_input / float(n_vocab)

    # Convert output to categorical format
    network_output = to_categorical(network_output, n_vocab)

    print("\n===== After Reshaping Data =====\n")
    print("normalized_input.shape:", normalized_input.shape)
    print("network_output.shape:", network_output.shape)

else:
    print(f"Error: The notes list must be longer than the sequence length ({sequence_length}).")



===== Explanation of Variables =====

notes: A list containing all the musical notes as strings.
Total number of notes in the score: 1833
Total unique note types in the score: 113
Unique note types: ['0', '0.3', '0.3.7', '0.4', '0.4.7', '0.5', '1.3.5.8', '1.3.8', '1.5.6.8', '1.5.8', '10', '10.1.3.6', '10.1.5', '10.11.3.6', '10.2.5', '11.3.6', '2', '2.5', '2.5.9', '2.6', '2.7', '3.5.6.10', '3.6.10', '3.6.8.11', '3.7', '3.7.10', '3.9', '4.5', '4.7', '4.9', '5', '5.10', '5.8', '5.8.0', '5.8.10.1', '5.9', '5.9.0', '7', '7.0', '7.10', '7.10.2', '7.11.2', '7.9', '8.0', '9', '9.0', '9.0.4', '9.2', 'A2', 'A3', 'A4', 'A5', 'A6', 'B-2', 'B-3', 'B-4', 'B-5', 'B0', 'B1', 'B2', 'B3', 'B4', 'B5', 'C#1', 'C#2', 'C#3', 'C#4', 'C#5', 'C#6', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'D2', 'D3', 'D4', 'D5', 'D6', 'E-1', 'E-2', 'E-3', 'E-4', 'E-5', 'E-6', 'E-7', 'E2', 'E3', 'E4', 'E5', 'E6', 'F#2', 'F#3', 'F#4', 'F#5', 'F#6', 'F2', 'F3', 'F4', 'F5', 'F6', 'G#1', 'G#2', 'G#3', 'G#4', 'G#5', 'G1', 'G2', 'G3', 'G

### Create the structure of a neural network 
### LSTM

In [6]:
"""
Create the structure of the neural network using LSTM layers.

This model is designed for sequence prediction tasks, leveraging the strengths of
LSTM layers for processing sequential data like music notes and chords.
"""

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Activation, BatchNormalization
from tensorflow.keras.optimizers import RMSprop

# Define the model
model = Sequential()

# Add the first LSTM layer with return_sequences=True for stacked LSTM
model.add(LSTM(
    512,  # Number of units in the LSTM layer
    input_shape=(normalized_input.shape[1], normalized_input.shape[2]),  # Input shape
    recurrent_dropout=0.1,  # Dropout for recurrent connections
    return_sequences=True  # Return sequences for stacking LSTM layers
))

# Add the second LSTM layer
model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.1))

# Add the third LSTM layer (no return_sequences since it's the last LSTM layer)
model.add(LSTM(512))

# Add Batch Normalization
model.add(BatchNormalization())

# Add a Dropout layer to reduce overfitting
model.add(Dropout(0.1))

# Add a Dense layer with 256 units and ReLU activation
model.add(Dense(256))
model.add(Activation('relu'))

# Add Batch Normalization
model.add(BatchNormalization())

# Add another Dropout layer
model.add(Dropout(0.1))

# Add the output layer with softmax activation
model.add(Dense(n_vocab))  # n_vocab is the number of unique notes/chords
model.add(Activation('softmax'))

# Compile the model with categorical crossentropy loss and RMSprop optimizer
model.compile(
    loss='categorical_crossentropy', 
    optimizer=RMSprop(learning_rate=0.001)  # Specify learning rate
)

# Display the model summary
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                   │ (None, 100, 512)       │     1,052,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 100, 512)       │     2,099,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 512)            │     2,099,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 113)            │        29,041 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 113)            │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,414,513 (20.65 MB)

 Trainable params: 5,412,977 (20.65 MB)

 Non-trainable params: 1,536 (6.00 KB)

### Train a neural network

In [7]:
"""
Train the neural network for generating music sequences.

This process adjusts the weights of the model based on the provided input
and output, enabling it to learn patterns in the musical dataset. Only the best model
based on training loss will be saved.
"""

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Define a callback to save the best model based on training loss
callbacks = [
    # Save only the best model based on minimum loss
    ModelCheckpoint(
        filepath='best_model.keras',  # Filepath to save the best model in .keras format
        monitor='loss',               # Monitor training loss for improvement
        save_best_only=True,          # Save only the best model weights
        mode='min',                   # Minimize the monitored value (loss)
        verbose=1
    ),
    # Stop training early if the loss stagnates
    EarlyStopping(
        monitor='loss', 
        patience=10,                  # Wait for 10 epochs of no improvement
        restore_best_weights=True     # Load the best weights when stopping
    )
]

# Train the model
history = model.fit(
    normalized_input,  # Input data
    network_output,    # Expected output
    epochs=50,         # Total number of training epochs
    batch_size=128,    # Size of each training batch
    callbacks=callbacks,  # Attach callbacks
    verbose=1          # Print progress during training
)

print("Training complete! The best model has been saved as 'best_model.keras'.")


Epoch 1/50


I0000 00:00:1733149582.782620   64003 cuda_dnn.cc:529] Loaded cuDNN version 90300


14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step - loss: 5.2066
Epoch 1: loss improved from inf to 5.01218, saving model to best_model.keras
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 169ms/step - loss: 5.1936
Epoch 2/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - loss: 4.4377
Epoch 2: loss improved from 5.01218 to 4.38788, saving model to best_model.keras
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 165ms/step - loss: 4.4344
Epoch 3/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - loss: 4.2987
Epoch 3: loss improved from 4.38788 to 4.27358, saving model to best_model.keras
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 160ms/step - loss: 4.2971
Epoch 4/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step - loss: 4.0477
Epoch 4: loss improved from 4.27358 to 4.09130, saving model to best_model.keras
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 164ms/step - loss: 4.0506
Epoch 5/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step - loss: 4.0876
Epoch 5: loss improved from 4.09130 to 4.03529, saving model to best_model.keras
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 167ms/step - loss: 4.

Based on the selected note starting point, predict the next note from the neural network and generate the score

In [8]:
"""
Generate music based on a starting sequence using the trained neural network.

This process uses the model to predict the next notes and constructs
a new sequence that can be converted into a MIDI file.
"""

import random

# Choose a random sequence from network_input as the starting point for generation
start = numpy.random.randint(0, len(network_input) - 1)
pattern = network_input[start]

# Map integers back to their corresponding notes/chords
int_to_note = {number: note for number, note in enumerate(pitchnames)}

# Store the generated sequence
prediction_output = []

print("Generating notes...")

# Generate a sequence of notes (adjust the range for sequence length)
for note_index in range(250):  # Generate more notes for richer output
    # Prepare the input for prediction
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)  # Normalize input

    # Predict probabilities for the next note
    prediction = model.predict(prediction_input, verbose=0)

    # Add some randomness to predictions for creativity
    top_indices = numpy.argsort(prediction[0])[-3:]  # Pick the top 3 predictions
    index = random.choices(top_indices, weights=prediction[0][top_indices])[0]

    # Map the predicted index to the corresponding note
    result = int_to_note[index]
    prediction_output.append(result)

    print(f"Note {note_index}: {result}")

    # Shift the prediction window and append the new note
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

print("Note generation complete!")


Generating notes...
Note 0: A4
Note 1: B4
Note 2: B4
Note 3: G#4
Note 4: G#4
Note 5: A4
Note 6: 5.9
Note 7: 5.9
Note 8: 5.9
Note 9: 5.9
Note 10: 5.9
Note 11: 5.9
Note 12: 5.9
Note 13: 4.5
Note 14: 4.5
Note 15: 5.9
Note 16: 4.5
Note 17: 4.9
Note 18: 4.5
Note 19: 4.5
Note 20: 5.9
Note 21: 5.9
Note 22: 5.9
Note 23: 5.9
Note 24: 4.5
Note 25: 5.9
Note 26: 5.9
Note 27: 1.5.6.8
Note 28: 5.9
Note 29: 1.5.6.8
Note 30: 1.5.6.8
Note 31: 0
Note 32: 0.4.7
Note 33: 0.4.7
Note 34: B5
Note 35: 3.6.10
Note 36: 10
Note 37: 10
Note 38: 0
Note 39: 0
Note 40: 0.4.7
Note 41: 0.4.7
Note 42: 0.4.7
Note 43: 0.4.7
Note 44: 0.4.7
Note 45: 0.4.7
Note 46: 0.4.7
Note 47: 0
Note 48: 0.4.7
Note 49: 5.9.0
Note 50: 5.9.0
Note 51: 5.9.0
Note 52: 5.9.0
Note 53: 5.9.0
Note 54: 5.9.0
Note 55: 5.9.0
Note 56: 1.5.8
Note 57: 0
Note 58: C#6
Note 59: 1.5.8
Note 60: 1.5.8
Note 61: 1.5.8
Note 62: 1.5.8
Note 63: 1.5.8
Note 64: 1.5.6.8
Note 65: 1.5.8
Note 66: 10
Note 67: E-7
Note 68: 1.5.8
Note 69: E3
Note 70: E3
Note 71: E3
Note 7

Convert predicted output to notes and create a MIDI file from the notes

In [9]:
"""
Convert the predicted output into a MIDI file.

This script takes the generated sequence of notes and chords, creates
corresponding MIDI objects, and saves them as a MIDI file.
"""

from music21 import stream, note, chord, instrument

# Initialize variables for MIDI creation
offset = 0  # Time spacing between notes/chords
output_notes = []

# Convert the predicted patterns into notes and chords
for pattern in prediction_output:
    # If the pattern represents a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            try:
                # Convert the note number into a Note object
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            except ValueError:
                print(f"Skipped invalid note: {current_note}")
        # Create a Chord object from the notes
        if notes:
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
    # If the pattern represents a single note
    else:
        try:
            # Convert the pattern into a Note object
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        except Exception as e:
            print(f"Error creating note '{pattern}': {e}")

    # Increment the offset for spacing
    offset += 0.5

# Create a music21 stream from the generated notes and chords
midi_stream = stream.Stream(output_notes)

# Save the stream as a MIDI file
output_filename = 'generated_music.mid'
try:
    midi_stream.write('midi', fp=output_filename)
    print(f"MIDI file successfully created: {output_filename}")
except Exception as e:
    print(f"Error writing MIDI file: {e}")


MIDI file successfully created: generated_music.mid
